In [44]:
from random import randint
import numpy as np
from keras.models import model_from_yaml


In [45]:
with open("hazes.txt") as corpus_file:
    corpus = corpus_file.read()
    corpus = corpus.lower()
print("Loaded a corpus of {0} characters".format(len(corpus)))

Loaded a corpus of 41701 characters


In [46]:
# Get a unique identifier for each char in the corpus, then make some dicts to ease encoding and decoding
chars = sorted(list(set(corpus)))
num_chars = len(chars)
encoding = {c: i for i, c in enumerate(chars)}
decoding = {i: c for i, c in enumerate(chars)}

In [47]:
# it slices, it dices, it makes julienned datasets!
# chop up our data into X and y, slice into roughly (num_chars / skip) overlapping 'sentences'
# of length sentence_length, and encode the chars
sentence_length = 20
corpus_length = len(corpus)
skip = 1
X_data = []
y_data = []

for i in range (0, len(corpus) - sentence_length, skip):
    sentence = corpus[i:i + sentence_length]
    next_char = corpus[i + sentence_length]
    X_data.append([encoding[char] for char in sentence])
    y_data.append(encoding[next_char])

In [48]:
num_sentences = len(X_data)

In [49]:
print("Vectorizing X and y...")
X = np.zeros((num_sentences, sentence_length, num_chars), dtype=np.bool)
y = np.zeros((num_sentences, num_chars), dtype=np.bool)
for i, sentence in enumerate(X_data):
    for t, encoded_char in enumerate(sentence):
        X[i, t, encoded_char] = 1
    y[i, y_data[i]] = 1

Vectorizing X and y...


In [50]:
# Build our network from loaded architecture and weights
with open('model.yaml') as model_file:
    architecture = model_file.read()
model = model_from_yaml(architecture)
model.load_weights('weights-20.hdf5')
model.compile(loss='categorical_crossentropy', optimizer='adam')


In [51]:
def sample(preds, temperature=1.0):
  if temperature <= 0:
    return np.argmax(preds)
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [52]:
# Functions

def generate(seed_pattern):
        X = np.zeros((1, sentence_length, num_chars), dtype=np.bool)
        #print(X.shape)
        for i, character in enumerate(seed_pattern):
            X[0, i, encoding[character]] = 1

        generated_text = ""
        for i in range(500):
            # even de temperatuur toevoegen.
            prediction = sample(model.predict(X, verbose=0)[0],0.3)
            generated_text += decoding[prediction]

            activations = np.zeros((1, 1, num_chars), dtype=np.bool)
            activations[0, 0, prediction] = 1
            X = np.concatenate((X[:, 1:, :], activations), axis=1)

        return generated_text

def make_seed(seed_phrase=""):
        if seed_phrase:
            phrase_length = len(seed_phrase)
            pattern = ""
            for i in range (0, sentence_length):
                pattern += seed_phrase[i % phrase_length]
        else:
            seed = randint(0, corpus_length - sentence_length)
            pattern = corpus[seed:seed + sentence_length]

        return pattern

In [54]:
# Set seed and generate text

seed = make_seed('gaat het straks nog sneeuwen')
print(seed)
txt =  generate(seed)
print(txt)


gaat het straks nog 
sne uwer
want voor die kleine heb ik een slee gemaakt
dan zit ik van mijn leen zeg met m'n kinder dan zel
ik ben toen
zonder je bent gezel van het mer geaan
maar ik deed een ander toch geen pijn
een keer in mijn leven is het mis gegaan
maar ik deed een ander toch geen pijn
een keer in mijn leven is het mis gegaan
maar ik deed een ander toch geen pijn
een keer in mijn leven is het mis gegaan
nee, ik wilde toen nooit meer bij je zijn

ik zit hier nu al uren, door het raam kijk ik op straat
ik weet
